In [1]:
import os
from dotenv import load_dotenv
import replicate

load_dotenv()

REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [2]:
input = "https://github.com/dcolinmorgan/modegen-ai/blob/master/Photo5623.jpg?raw=true"
fit = "https://github.com/dcolinmorgan/modegen-ai/blob/master/fit_pose.jpg?raw=true"

## this is threads perspective -- check rabbit for queue system

In [10]:
# sam = replicate.models.get("meta/sam-2").versions.get("fe97b453a6455861e3bac769b441ca1f1086110da7466dbb65cf1eecfd60dc83")
cn = replicate.models.get("lucataco/sdxl-controlnet-openpose").versions.get("d63e0b238b2d963d90348e2dad19830fbe372a7a43d90d234b2b63cae76d4397")
cn_lora = replicate.models.get("fofr/sdxl-multi-controlnet-lora").versions.get("89eb212b3d1366a83e949c12a4b45dfe6b6b313b594cb8268e864931ac9ffb16")
esrgan = replicate.models.get("nightmareai/real-esrgan").versions.get("42fed1c4974146d4d2414e2be2c5277c7fcf05fcc3a73abf41610695738c1d7b")

# mask = replicate.predictions.create(version=sam, input={"image": fit,
#                                                 "use_m2m": True,
#                                                 "points_per_side": 32,
#                                                 "pred_iou_thresh": 0.88,
#                                                 "stability_score_thresh": 0.5})
# mask.wait()

# image = replicate.predictions.create(version=cn, input={"image": fit}) #mask.output['individual_masks'][0]})
# image.wait()

out = replicate.predictions.create(version=cn_lora,
    input={
        "image": input,
        "width": 768,
        "height": 768,
        "prompt": "A photo of this woman in the red dress",
        "refine": "no_refiner",
        "scheduler": "K_EULER",
        "lora_scale": 0.8,
        "num_outputs": 1,
        "controlnet_1": "none",
        "controlnet_2": "none",
        "controlnet_3": "none",
        "lora_weights": "https://replicate.delivery/pbxt/hKhpVe6O7EwXNCiWORev3OEDRCoWeMlqZMLQDEvwDyHV3hvjA/trained_model.tar",
        "guidance_scale": 7.5,
        "apply_watermark": False,
        "negative_prompt": "wheel, shoe",
        "prompt_strength": 0.8,
        "sizing_strategy": "width_height",
        "controlnet_1_end": 1,
        "controlnet_2_end": 1,
        "controlnet_3_end": 1,
        "controlnet_1_image": image.output, 
        "controlnet_1_start": 0,
        "controlnet_2_start": 0,
        "controlnet_3_start": 0,
        "num_inference_steps": 30,
        "controlnet_1_conditioning_scale": 0.8,
        "controlnet_2_conditioning_scale": 0.8,
        "controlnet_3_conditioning_scale": 0.75
    }
)
out.wait()

upscaled_image = replicate.predictions.create(version=esrgan, input={"image": out.output[0]})
upscaled_image.wait()

In [20]:
image.output

'https://replicate.delivery/pbxt/HaE9gBqf8qXek0SbLDEiQsKNj7rRG8Hyipv4w4Ka5KbT29QTA/output.png'

In [21]:
from IPython.display import Image
Image(url=image.output[0])

In [ ]:
from urllib.request import urlretrieve

urlretrieve(upscaled_image.output[0], "/tmp/out.png")

# Run a model
You can run any public model on Replicate from your Python code. The following example runs [stability-ai/stable-diffusion](https://replicate.com/stability-ai/stable-diffusion):

In [ ]:
import replicate

output = replicate.run(
  "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
  input={"prompt": "an beagle on the beach, pointillism"}
)

output

In [ ]:
from IPython.display import Image
Image(url=output[0])

Some models receive images as inputs. To pass a file as an input, use a file handle or URL:

In [ ]:
# image = open("mystery.jpg", "rb")
# or...
# image = "https://example.com/mystery.jpg"
#
# for this example, let's use the URL from the previous prediction:
image = output[0]

replicate.run(
  "andreasjansson/blip-2:4b32258c42e9efd4288bb9910bc532a69727f9acd26aa08e175713a0a857a608",
  input={"image": output[0], "prompt": "what's in this picture?"}
)

# Run a model in the background
You can start a model and run it in the background:

In [ ]:
model = replicate.models.get("meta/musicgen")
version = model.versions.get("b05b1dff1d8c6dc63d14b0cdb42135378dcb87f6373b0d3d341ede46e59e2b38")

prediction = replicate.predictions.create(
    version=version,
    input={"prompt":"Edo25 major g melodies that sound triumphant and cinematic. Leading up to a crescendo that resolves in a 9th harmonic"})

In [ ]:
prediction

In [ ]:
prediction.status

In [ ]:
dict(prediction).keys()

In [ ]:
prediction.wait()
prediction.reload()

In [ ]:
prediction.status

In [ ]:
import IPython
IPython.display.Audio(prediction.output)

# Run a model in the background and get a webhook

You can run a model and get a webhook when it completes, instead of waiting for it to finish.

If you're working locally, we'd recommend using [ngrok](https://ngrok.com/download) for debugging webhooks. It allows you to tunnel your localhost to a public domain.

In [ ]:
model = replicate.models.get("jd7h/photomaker")
version = model.versions.get("79b21a43649e0d08a7cbf9b553420047456de4492efb1a7c73d500c1c8812fc6")
prediction = replicate.predictions.create(
    version=version,
    input={
        "seed": 1143488585,
        "prompt": "A photo of a scientist winning the Nobel prize img",
        "num_steps": 50,
        "style_name": "Photographic (Default)",
        "input_image": "https://replicate.delivery/pbxt/KF73Yow3WJw7ebPRq1tPohZXN0bjoAjbNcXiJzeineGGAv1A/newton_0.jpg",
        "num_outputs": 1,
        "guidance_scale": 5,
        "negative_prompt": "nsfw, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry",
        "style_strength_ratio": 20
    },
    webhook="https://example.com/your-webhook",
    webhook_events_filter=["completed"]
)

# Get streaming output from a running model
Some models stream output as the model is running. They will return an iterator, and you can iterate over that output:

In [ ]:
iterator = replicate.run(
  "mistralai/mixtral-8x7b-instruct-v0.1",
  input={"prompt": "Who was Dolly the sheep?"},
)
for text in iterator:
      print(text)

# Cancel a prediction
You can cancel a running prediction:

In [ ]:
model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": "https://replicate.delivery/pbxt/JvLi9smWKKDfQpylBYosqQRfPKZPntuAziesp0VuPjidq61n/rocket.png",
      "video_length": "14_frames_with_svd",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 6
    })

In [ ]:
prediction.status

In [ ]:
prediction.cancel()

In [ ]:
prediction.reload()

In [ ]:
prediction.status

In [ ]:
predictions = replicate.predictions.list()

[p.status for p in predictions[:10]]

# Load output files
Output files are returned as HTTPS URLs. You can load an output file as a buffer:

In [ ]:
from urllib.request import urlretrieve

model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": "https://replicate.delivery/pbxt/JvLi9smWKKDfQpylBYosqQRfPKZPntuAziesp0VuPjidq61n/rocket.png",
      "video_length": "14_frames_with_svd",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 6
    })
prediction.wait()

urlretrieve(prediction.output, "/tmp/out.mp4")

In [ ]:
from IPython.display import Video

Video(prediction.output)

# Next steps

- Read the [full Python client documentation on GitHub](https://github.com/replicate/replicate-python#readme).

- Explore our collection of hosted [models](https://replicate.com/explore)

- Note that you can also run models with the raw HTTP API. Refer to the [HTTP API reference](https://replicate.com/docs/reference/http) for more details.